In [1]:
import os

from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor
from langchain import hub, LLMMathChain
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

/Users/niaki/Workspace/cocktailr-backend/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.0,
    max_tokens=None,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

In [3]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def get_most_similar_cocktail(cocktail: str) -> str:
    """Returns the most similar cocktail to the input cocktail."""
    return "Margarita"


get_word_length.invoke("abc")



3

In [4]:
tools = [get_word_length, get_most_similar_cocktail]

In [5]:
agent_prompt = hub.pull("mikechan/gemini")
agent_prompt.template

'Your name is “Kasper”. You are world class ai personal assistant computer that created by Mike Chan(A independent AI engineer). You are highly advanced intelligent. you are very helpful and good to provide information and solve users problem. Kasper is a AELM model (Auto Execution Language Model).\n\n“ Kasper” is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, “ Kasper” is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\n“ Kasper” is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, “ Kasper” is

In [6]:
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [7]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [8]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [9]:
agent_executor.invoke({"input": "what's the most similar cocktail to white russian? and how many letters does that cocktail's name have"})["output"]



> Entering new AgentExecutor chain...


/Users/niaki/Workspace/cocktailr-backend/.venv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? Yes
Action: get_most_similar_cocktail
Action Input: white russianMargarita

/Users/niaki/Workspace/cocktailr-backend/.venv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Do I need to use a tool? Yes
Action: get_word_length
Action Input: Margarita9

/Users/niaki/Workspace/cocktailr-backend/.venv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Do I need to use a tool? No
Final Answer: The most similar cocktail to a White Russian is a Margarita, and it has 9 letters in its name.

> Finished chain.


'The most similar cocktail to a White Russian is a Margarita, and it has 9 letters in its name.'